In [1]:
import tensorflow as tf
import numpy as np
from glob import glob
import matplotlib.pyplot as plt
import os
import cv2
import math

import utils

mnist_path = 'data/mnist-zoomed-to-bounds.npy'
model_path = 'data/model'

2021-08-24 19:04:10.492009: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-08-24 19:04:10.492030: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
/home/dan/code/tf-digit-recognition-playground/utils.py:17: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if image.shape[2] is 3:


In [2]:
img = cv2.imread('/home/dan/Downloads/poule-pngs/poule-011.png', cv2.IMREAD_UNCHANGED)
img = utils.deskew(img, debug=True)

img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
th, thresh = cv2.threshold(img_gray, 127, 255, cv2.THRESH_BINARY)
thresh = 255 - thresh

blur=cv2.GaussianBlur(thresh, (5, 5), 1)
# canny=cv2.Canny(blur, 5, 25)

contours, hierarchy = cv2.findContours(
    image=blur, 
    mode=cv2.RETR_TREE,
    method=cv2.CHAIN_APPROX_NONE
)
print(len(contours))

# This is too naive. One way to improve could be using width/height ratio to filter
# 1/1 would be square = group stage. 2/1 would be landscape rect = match entry, etc. 
filtered = []
for c in contours:
    o = 20
    area = cv2.contourArea(c)
    if area < 1000 or area > 5000: continue
    perimeter = cv2.arcLength(c, True)
    approximation = cv2.approxPolyDP(c, 0.02 * perimeter, True)
    if len(approximation) > 100: continue
    x, y, w, h = cv2.boundingRect(approximation)
    if w >= h + o or w <= h - o: continue
    filtered.append([x, y, x + w, y + h])

# Naive duplicate filtering... Is there a better openCV'nic way?
rects = []
for r in filtered:
    rel_tol = 0.02
    existing = [er for er in rects if math.isclose(r[0], er[0], rel_tol=rel_tol) and math.isclose(r[1], er[1], rel_tol=rel_tol)]
    if len(existing): continue
    rects.append(r)
    cv2.rectangle(img, (r[0], r[1]), (r[2], r[3]), (0, 255, 0), 3)

rects = np.asarray(rects)
    
# Sort from left to right top to bottom
# print('hai', rects[rects[:1].argsort()])
print(rects)
print(np.asarray(rects).shape)

thresh = np.asarray(thresh)
print(thresh.shape)

# thresh = thresh.reshape(*thresh.shape, 1)
thresh = np.expand_dims(thresh, axis=2)

thresh = thresh / 255

print(thresh.shape)

plt.figure(figsize=(25, 25))
plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
plt.show()

black_blur image.shape: (3508, 2481, 3)
deskew() Found angle: 0.0


error: OpenCV(4.5.3) /tmp/pip-req-build-afu9cjzs/opencv/modules/imgproc/src/color.simd_helpers.hpp:92: error: (-2:Unspecified error) in function 'cv::impl::{anonymous}::CvtHelper<VScn, VDcn, VDepth, sizePolicy>::CvtHelper(cv::InputArray, cv::OutputArray, int) [with VScn = cv::impl::{anonymous}::Set<3, 4>; VDcn = cv::impl::{anonymous}::Set<1>; VDepth = cv::impl::{anonymous}::Set<0, 2, 5>; cv::impl::{anonymous}::SizePolicy sizePolicy = cv::impl::<unnamed>::NONE; cv::InputArray = const cv::_InputArray&; cv::OutputArray = const cv::_OutputArray&]'
> Invalid number of channels in input image:
>     'VScn::contains(scn)'
> where
>     'scn' is 1


In [ ]:
physical_devices = tf.config.list_physical_devices('GPU')
physical_devices

In [ ]:
# Use cached augmented data if it exists
if os.path.isfile(mnist_path) is False:
    print("Normalizing mnist data set")
    mnist = tf.keras.datasets.mnist

    (x_train, y_train), (x_test, y_test) = mnist.load_data()
    x_train = x_train / 255.0
    x_test  = x_test  / 255.0
    
    x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
    x_test  = x_test.reshape(x_test.shape[0], 28, 28, 1)

    x_train = np.array([utils.zoom_to_bounds(img_array) for img_array in x_train])
    x_test  = np.array([utils.zoom_to_bounds(img_array) for img_array in x_test])

    to_save =  np.array( ([x_train, y_train], [x_test, y_test]) )
    np.save(mnist_path, to_save)
else:
    print("Using cached mnist data set")
    (x_train, y_train), (x_test, y_test) = np.load(mnist_path, allow_pickle=True)

In [ ]:
# Not overwriting if already exists!
if os.path.isdir(model_path) is False:
    print("Training model")
    model = tf.keras.models.Sequential([
      tf.keras.layers.Flatten(input_shape=(28, 28)),
      tf.keras.layers.Dense(128, activation='relu'),
      tf.keras.layers.Dropout(0.2),
      tf.keras.layers.Dense(10)
    ])
    model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
    model.fit(x_train, y_train, epochs=12)
    model.save(model_path)
else:
    print("Loading cached model")
    model = tf.keras.models.load_model(model_path)
    
model.evaluate(x_test,  y_test, verbose=2)

In [ ]:
probability_model = tf.keras.Sequential([
  model,
  tf.keras.layers.Softmax()
])

In [ ]:
pngs = sorted(glob("data/pngs/*.png"))
def load_image(path):
    image     = tf.keras.preprocessing.image.load_img(path, color_mode="grayscale", target_size=(28,28), interpolation='nearest')
    input_arr = tf.keras.preprocessing.image.img_to_array(image)
    return input_arr / 255

actual_data = [load_image(png) for png in pngs]
actual_data = np.array(actual_data)
actual_data.shape

In [ ]:
group_stage = []
for r in rects:
    cropped = tf.image.crop_to_bounding_box(
        blur.reshape(*blur.shape, 1), r[1], r[0], r[3] - r[1], r[2] - r[0]
    )
    cropped = tf.image.central_crop(
        cropped, 0.8
    )
    resized = tf.image.resize(
        cropped, (28,28), method=tf.image.ResizeMethod.BILINEAR, preserve_aspect_ratio=True
    )
    
    if len(np.nonzero(resized)[0]) < 3: continue
    
    plt.imshow(resized, cmap="gray")
    plt.show()
    group_stage.append(resized)

In [ ]:
plt.figure(figsize=(25, 25))
plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
plt.show()
data = np.array([utils.zoom_to_bounds(img_array) for img_array in group_stage])
predictions = probability_model(data) * 100
for i, d in enumerate(data):
    fig, (ax1, ax2) = plt.subplots(ncols=2, figsize=(10, 5))
    ax1.imshow(d, cmap="gray")
    ax2.bar(x=range(0, 10), height=predictions[i], tick_label=range(0, 10))
    ax2.set(ylabel="certainty", ylim=(0, 100))
